In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Basic Feature Engineering - Fraud Detection\n",
    "\n",
    "**Objective**: Create fundamental features for fraud detection\n",
    "**Author**: Data Science Team\n",
    "**Date**: 2025-10-17\n",
    "**Version**: 1.0\n",
    "\n",
    "## Table of Contents\n",
    "1. [Setup and Data Loading](#1-setup-and-data-loading)\n",
    "2. [Transaction Features](#2-transaction-features)\n",
    "3. [User Features](#3-user-features)\n",
    "4. [Merchant Features](#4-merchant-features)\n",
    "5. [Temporal Features](#5-temporal-features)\n",
    "6. [Feature Validation](#6-feature-validation)\n",
    "7. [Feature Export](#7-feature-export)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Data Loading"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from datetime import datetime, timedelta\n",
    "from sklearn.preprocessing import StandardScaler, LabelEncoder\n",
    "from sklearn.feature_selection import mutual_info_classif\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Custom utilities\n",
    "import sys\n",
    "sys.path.append('../utils')\n",
    "from feature_utils import create_velocity_features, create_behavioral_features\n",
    "\n",
    "print(\"🚀 Feature Engineering Environment Ready!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load processed data from previous notebook\n",
    "df = pd.read_csv('../outputs/processed_transactions.csv', parse_dates=['timestamp'])\n",
    "print(f\"📊 Data loaded: {df.shape}\")\n",
    "print(f\"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}\")\n",
    "print(f\"Fraud rate: {df['is_fraud'].mean():.2%}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Transaction Features"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_transaction_features(df):\n",
    "    \"\"\"Create basic transaction-level features\"\"\"\n",
    "    \n",
    "    # Amount-based features\n",
    "    df['amount_log'] = np.log1p(df['amount'])\n",
    "    df['amount_squared'] = df['amount'] ** 2\n",
    "    df['amount_sqrt'] = np.sqrt(df['amount'])\n",
    "    \n",
    "    # Round number indicators\n",
    "    df['is_round_amount'] = (df['amount'] % 1 == 0).astype(int)\n",
    "    df['is_round_10'] = (df['amount'] % 10 == 0).astype(int)\n",
    "    df['is_round_100'] = (df['amount'] % 100 == 0).astype(int)\n",
    "    \n",
    "    # Amount percentiles (within each merchant)\n",
    "    df['amount_percentile_merchant'] = df.groupby('merchant_id')['amount'].rank(pct=True)\n",
    "    \n",
    "    # Payment method encoding\n",
    "    le_payment = LabelEncoder()\n",
    "    df['payment_method_encoded'] = le_payment.fit_transform(df['payment_method'])\n",
    "    \n",
    "    # Payment method risk (based on fraud rates)\n",
    "    payment_fraud_rates = df.groupby('payment_method')['is_fraud'].mean()\n",
    "    df['payment_method_risk'] = df['payment_method'].map(payment_fraud_rates)\n",
    "    \n",
    "    return df\n",
    "\n",
    "# Apply transaction features\n",
    "df = create_transaction_features(df)\n",
    "print(\"✅ Transaction features created\")\n",
    "print(f\"New feature count: {df.shape[1] - 10}\")"  # Assuming 10 original columns"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. User Features"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_user_features(df):\n",
    "    \"\"\"Create user-level aggregated features\"\"\"\n",
    "    \n",
    "    # User transaction history (up to current transaction)\n",
    "    df_sorted = df.sort_values(['user_id', 'timestamp'])\n",
    "    \n",
    "    # User transaction count (cumulative)\n",
    "    df_sorted['user_transaction_count'] = df_sorted.groupby('user_id').cumcount() + 1\n",
    "    \n",
    "    # User total amount spent (cumulative)\n",
    "    df_sorted['user_total_amount'] = df_sorted.groupby('user_id')['amount'].cumsum()\n",
    "    \n",
    "    # User average transaction amount\n",
    "    df_sorted['user_avg_amount'] = (df_sorted['user_total_amount'] / \n",
    "                                   df_sorted['user_transaction_count'])\n",
    "    \n",
    "    # User spending patterns\n",
    "    df_sorted['amount_vs_user_avg'] = (df_sorted['amount'] / \n",
    "                                      df_sorted['user_avg_amount'])\n",
    "    \n",
    "    # Days since first transaction for this user\n",
    "    df_sorted['user_first_transaction'] = df_sorted.groupby('user_id')['timestamp'].transform('first')\n",
    "    df_sorted['days_since_first_transaction'] = (\n",
    "        df_sorted['timestamp'] - df_sorted['user_first_transaction']\n",
    "    ).dt.days\n",
    "    \n",
    "    # User merchant diversity (up to current transaction)\n",
    "    def rolling_nunique(series):\n",
    "        return series.expanding().nunique()\n",
    "    \n",
    "    df_sorted['user_merchant_diversity'] = (\n",
    "        df_sorted.groupby('user_id')['merchant_id'].transform(rolling_nunique)\n",
    "    )\n",
    "    \n",
    "    # User payment method diversity\n",
    "    df_sorted['user_payment_diversity'] = (\n",
    "        df_sorted.groupby('user_id')['payment_method'].transform(rolling_nunique)\n",
    "    )\n",
    "    \n",
    "    return df_sorted.sort_index()  # Return to original order\n",
    "\n",
    "# Apply user features\n",
    "df = create_user_features(df)\n",
    "print(\"✅ User features created\")\n",
    "print(f\"Total features: {df.shape[1]}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Merchant Features"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_merchant_features(df):\n",
    "    \"\"\"Create merchant-level features\"\"\"\n",
    "    \n",
    "    # Merchant risk score (fraud rate)\n",
    "    merchant_fraud_rates = df.groupby('merchant_id')['is_fraud'].mean()\n",
    "    df['merchant_fraud_rate'] = df['merchant_id'].map(merchant_fraud_rates)\n",
    "    \n",
    "    # Merchant transaction volume\n",
    "    merchant_volume = df.groupby('merchant_id').size()\n",
    "    df['merchant_transaction_volume'] = df['merchant_id'].map(merchant_volume)\n",
    "    \n",
    "    # Merchant average transaction amount\n",
    "    merchant_avg_amount = df.groupby('merchant_id')['amount'].mean()\n",
    "    df['merchant_avg_amount'] = df['merchant_id'].map(merchant_avg_amount)\n",
    "    \n",
    "    # Amount vs merchant average\n",
    "    df['amount_vs_merchant_avg'] = df['amount'] / df['merchant_avg_amount']\n",
    "    \n",
    "    # Merchant customer diversity\n",
    "    merchant_customer_count = df.groupby('merchant_id')['user_id'].nunique()\n",
    "    df['merchant_customer_diversity'] = df['merchant_id'].map(merchant_customer_count)\n",
    "    \n",
    "    # Merchant payment method diversity\n",
    "    merchant_payment_diversity = df.groupby('merchant_id')['payment_method'].nunique()\n",
    "    df['merchant_payment_diversity'] = df['merchant_id'].map(merchant_payment_diversity)\n",
    "    \n",
    "    return df\n",
    "\n",
    "# Apply merchant features\n",
    "df = create_merchant_features(df)\n",
    "print(\"✅ Merchant features created\")\n",
    "print(f\"Total features: {df.shape[1]}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Temporal Features"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_temporal_features(df):\n",
    "    \"\"\"Create time-based features\"\"\"\n",
    "    \n",
    "    # Basic time features\n",
    "    df['hour'] = df['timestamp'].dt.hour\n",
    "    df['day_of_week'] = df['timestamp'].dt.dayofweek\n",
    "    df['day_of_month'] = df['timestamp'].dt.day\n",
    "    df['month'] = df['timestamp'].dt.month\n",
    "    df['quarter'] = df['timestamp'].dt.quarter\n",
    "    df['year'] = df['timestamp'].dt.year\n",
    "    \n",
    "    # Weekend indicator\n",
    "    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)\n",
    "    \n",
    "    # Business hours indicator\n",
    "    df['is_business_hours'] = ((df['hour'] >= 9) & (df['hour'] <= 17)).astype(int)\n",
    "    \n",
    "    # Late night transactions (high risk)\n",
    "    df['is_late_night'] = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)\n",
    "    \n",
    "    # Cyclical encoding for temporal features\n",
    "    # Hour (24-hour cycle)\n",
    "    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)\n",
    "    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)\n",
    "    \n",
    "    # Day of week (7-day cycle)\n",
    "    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)\n",
    "    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)\n",
    "    \n",
    "    # Month (12-month cycle)\n",
    "    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)\n",
    "    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)\n",
    "    \n",
    "    # Days since epoch (for trend)\n",
    "    epoch = pd.Timestamp('2020-01-01')\n",
    "    df['days_since_epoch'] = (df['timestamp'] - epoch).dt.days\n",
    "    \n",
    "    return df\n",
    "\n",
    "# Apply temporal features\n",
    "df = create_temporal_features(df)\n",
    "print(\"✅ Temporal features created\")\n",
    "print(f\"Total features: {df.shape[1]}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Feature Validation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get feature columns (excluding ID and target)\n",
    "feature_cols = [col for col in df.columns \n",
    "                if col not in ['user_id', 'merchant_id', 'timestamp', 'is_fraud', 'transaction_id']]\n",
    "\n",
    "print(f\"📊 FEATURE SUMMARY:\")\n",
    "print(f\"Total features created: {len(feature_cols)}\")\n",
    "print(f\"Dataset shape: {df.shape}\")\n",
    "\n",
    "# Check for missing values in features\n",
    "missing_features = df[feature_cols].isnull().sum()\n",
    "missing_features = missing_features[missing_features > 0]\n",
    "\n",
    "if len(missing_features) > 0:\n",
    "    print(f\"\\n⚠️ Features with missing values:\")\n",
    "    for feature, count in missing_features.items():\n",
    "        print(f\"   {feature}: {count} ({count/len(df)*100:.2f}%)\")\nelse:\n",
    "    print(f\"\\n✅ No missing values in features\")\n",
    "\n",
    "# Check for infinite values\n",
    "inf_features = []\n",
    "for col in feature_cols:\n",
    "    if np.isinf(df[col]).any():\n",
    "        inf_features.append(col)\n",
    "\n",
    "if inf_features:\n",
    "    print(f\"\\n⚠️ Features with infinite values: {inf_features}\")\n",
    "    # Replace infinite values with NaN, then fill with median\n",
    "    for col in inf_features:\n",
    "        df[col] = df[col].replace([np.inf, -np.inf], np.nan)\n",
    "        df[col] = df[col].fillna(df[col].median())\n",
    "    print(f\"✅ Infinite values handled\")\nelse:\n",
    "    print(f\"✅ No infinite values detected\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature importance analysis\n",
    "print(f\"\\n🔍 FEATURE IMPORTANCE ANALYSIS:\")\n",
    "print(\"=\"*40)\n",
    "\n",
    "# Calculate mutual information scores\n",
    "X = df[feature_cols]\n",
    "y = df['is_fraud']\n",
    "\n",
    "# Handle any remaining NaN values\n",
    "X = X.fillna(X.median())\n",
    "\n",
    "mi_scores = mutual_info_classif(X, y, random_state=42)\n",
    "feature_importance = pd.DataFrame({\n",
    "    'feature': feature_cols,\n",
    "    'mutual_info_score': mi_scores\n",
    "}).sort_values('mutual_info_score', ascending=False)\n",
    "\n",
    "# Display top features\n",
    "print(f\"\\nTop 15 most important features:\")\n",
    "display(feature_importance.head(15))\n",
    "\n",
    "# Visualize feature importance\n",
    "plt.figure(figsize=(12, 8))\n",
    "top_features = feature_importance.head(20)\n",
    "sns.barplot(data=top_features, x='mutual_info_score', y='feature')\n",
    "plt.title('Top 20 Features by Mutual Information Score', fontsize=14, fontweight='bold')\n",
    "plt.xlabel('Mutual Information Score')\n",
    "plt.ylabel('Feature')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature correlation analysis\n",
    "print(f\"\\n🔗 FEATURE CORRELATION ANALYSIS:\")\n",
    "print(\"=\"*35)\n",
    "\n",
    "# Calculate correlation matrix for top features\n",
    "top_feature_names = feature_importance.head(15)['feature'].tolist()\n",
    "corr_matrix = df[top_feature_names].corr()\n",
    "\n",
    "# Find highly correlated features\n",
    "high_corr_pairs = []\n",
    "for i in range(len(corr_matrix.columns)):\n",
    "    for j in range(i+1, len(corr_matrix.columns)):\n",
    "        if abs(corr_matrix.iloc[i, j]) > 0.8:\n",
    "            high_corr_pairs.append({\n",
    "                'feature1': corr_matrix.columns[i],\n",
    "                'feature2': corr_matrix.columns[j],\n",
    "                'correlation': corr_matrix.iloc[i, j]\n",
    "            })\n",
    "\n",
    "if high_corr_pairs:\n",
    "    print(f\"\\n⚠️ Highly correlated feature pairs (|r| > 0.8):\")\n",
    "    for pair in high_corr_pairs:\n",
    "        print(f\"   {pair['feature1']} <-> {pair['feature2']}: {pair['correlation']:.3f}\")\nelse:\n",
    "    print(f\"\\n✅ No highly correlated feature pairs detected\")\n",
    "\n",
    "# Visualize correlation heatmap\n",
    "plt.figure(figsize=(12, 10))\n",
    "mask = np.triu(np.ones_like(corr_matrix, dtype=bool))\n",
    "sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', \n",
    "            center=0, square=True, fmt='.2f', cbar_kws={\"shrink\": .8})\n",
    "plt.title('Feature Correlation Heatmap (Top 15 Features)', fontsize=14, fontweight='bold')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Feature Export"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Export feature engineering results\n",
    "print(f\"💾 EXPORTING RESULTS:\")\n",
    "print(\"=\"*20)\n",
    "\n",
    "# Create feature documentation\n",
    "feature_docs = {\n",
    "    'feature_name': [],\n",
    "    'feature_type': [],\n",
    "    'description': [],\n",
    "    'mutual_info_score': []\n",
    "}\n",
    "\n",
    "# Add feature descriptions\n",
    "feature_descriptions = {\n",
    "    'amount_log': 'Log-transformed transaction amount',\n",
    "    'amount_squared': 'Squared transaction amount',\n",
    "    'amount_sqrt': 'Square root of transaction amount',\n",
    "    'is_round_amount': 'Binary indicator for round amounts',\n",
    "    'is_round_10': 'Binary indicator for amounts divisible by 10',\n",
    "    'is_round_100': 'Binary indicator for amounts divisible by 100',\n",
    "    'amount_percentile_merchant': 'Amount percentile within merchant',\n",
    "    'payment_method_encoded': 'Label-encoded payment method',\n",
    "    'payment_method_risk': 'Payment method fraud rate',\n",
    "    'user_transaction_count': 'Cumulative user transaction count',\n",
    "    'user_total_amount': 'Cumulative user spending amount',\n",
    "    'user_avg_amount': 'User average transaction amount',\n",
    "    'amount_vs_user_avg': 'Current amount vs user average',\n",
    "    'days_since_first_transaction': 'Days since user first transaction',\n",
    "    'user_merchant_diversity': 'Number of unique merchants per user',\n",
    "    'user_payment_diversity': 'Number of unique payment methods per user',\n",
    "    'merchant_fraud_rate': 'Merchant historical fraud rate',\n",
    "    'merchant_transaction_volume': 'Total transactions per merchant',\n",
    "    'merchant_avg_amount': 'Merchant average transaction amount',\n",
    "    'amount_vs_merchant_avg': 'Current amount vs merchant average',\n",
    "    'merchant_customer_diversity': 'Number of unique customers per merchant',\n",
    "    'merchant_payment_diversity': 'Payment method diversity per merchant',\n",
    "    'hour': 'Hour of day (0-23)',\n",
    "    'day_of_week': 'Day of week (0-6)',\n",
    "    'is_weekend': 'Binary indicator for weekend',\n",
    "    'is_business_hours': 'Binary indicator for business hours',\n",
    "    'is_late_night': 'Binary indicator for late night hours',\n",
    "    'hour_sin': 'Sine encoding of hour',\n",
    "    'hour_cos': 'Cosine encoding of hour',\n",
    "    'day_of_week_sin': 'Sine encoding of day of week',\n",
    "    'day_of_week_cos': 'Cosine encoding of day of week',\n",
    "    'month_sin': 'Sine encoding of month',\n",
    "    'month_cos': 'Cosine encoding of month',\n",
    "    'days_since_epoch': 'Days since epoch for trend'\n",
    "}\n",
    "\n",
    "# Create feature documentation DataFrame\n",
    "for feature in feature_cols:\n",
    "    feature_docs['feature_name'].append(feature)\n",
    "    \n",
    "    # Determine feature type\n",
    "    if df[feature].dtype in ['int64', 'int32']:\n",
    "        feature_type = 'integer'\n",
    "    elif df[feature].dtype in ['float64', 'float32']:\n",
    "        feature_type = 'continuous'\n",
    "    else:\n",
    "        feature_type = 'categorical'\n",
    "    \n",
    "    feature_docs['feature_type'].append(feature_type)\n",
    "    feature_docs['description'].append(\n",
    "        feature_descriptions.get(feature, 'Auto-generated feature')\n",
    "    )\n",
    "    \n",
    "    # Add mutual information score\n",
    "    mi_score = feature_importance[\n",
    "        feature_importance['feature'] == feature\n",
    "    ]['mutual_info_score'].values\n",
    "    \n",
    "    feature_docs['mutual_info_score'].append(\n",
    "        mi_score[0] if len(mi_score) > 0 else 0\n",
    "    )\n",
    "\n",
    "feature_docs_df = pd.DataFrame(feature_docs)\n",
    "\n",
    "# Save feature documentation\n",
    "feature_docs_df.to_csv('../outputs/feature_documentation.csv', index=False)\n",
    "print(f\"✅ Feature documentation saved: ../outputs/feature_documentation.csv\")\n",
    "\n",
    "# Save feature importance\n",
    "feature_importance.to_csv('../outputs/feature_importance.csv', index=False)\n",
    "print(f\"✅ Feature importance saved: ../outputs/feature_importance.csv\")\n",
    "\n",
    "# Save processed dataset with features\n",
    "df.to_csv('../outputs/dataset_with_features.csv', index=False)\n",
    "print(f\"✅ Dataset with features saved: ../outputs/dataset_with_features.csv\")\n",
    "\n",
    "# Display summary\n",
    "print(f\"\\n📊 FEATURE ENGINEERING SUMMARY:\")\n",
    "print(f\"   • Total features created: {len(feature_cols)}\")\n",
    "print(f\"   • Dataset shape: {df.shape}\")\n",
    "print(f\"   • Top feature: {feature_importance.iloc[0]['feature']}\")\n",
    "print(f\"   • Top MI score: {feature_importance.iloc[0]['mutual_info_score']:.4f}\")\n",
    "print(f\"\\n🎯 Ready for model development!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "### Features Created\n",
    "1. **Transaction Features**: Amount transformations, round number indicators, percentiles\n",
    "2. **User Features**: Transaction history, spending patterns, diversity metrics\n",
    "3. **Merchant Features**: Risk scores, volume metrics, customer diversity\n",
    "4. **Temporal Features**: Time-based patterns with cyclical encoding\n",
    "\n",
    "### Key Insights\n",
    "- Created comprehensive feature set for fraud detection\n",
    "- Identified most predictive features using mutual information\n",
    "- Handled data quality issues (missing values, infinite values)\n",
    "- Created proper feature documentation for reproducibility\n",
    "\n",
    "### Next Steps\n",
    "1. **Velocity Features**: Time-based transaction frequency features\n",
    "2. **Behavioral Features**: Advanced user behavior patterns\n",
    "3. **Network Features**: Graph-based relationship features\n",
    "4. **Model Development**: Use features for baseline model training\n",
    "\n",
    "---\n",
    "\n",
    "**Next Notebook**: `02_velocity_features.ipynb` - Time-based velocity feature engineering"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4    
}

NameError: name 'null' is not defined